In [1]:
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

In [5]:
import os
import sys
sys.path.append("/home/rosario/xmimsim/xsimspe")
from glob import glob
import spectra_utils as su
from collections import namedtuple
import pandas as pd

xrdata = pd.read_csv('/home/rosario/xmimsim/xsimspe/Xraydata.csv')
datadir = "/home/rosario/xmimsim/tests/reflay_th/outdata"

def get_ewf(info):
    index = 0
    ewf = []
    elements = info.elements.split('-')
    while index < len(elements):
        ewf.append((elements[index], elements[index + 1]))
        index += 2
    return ewf

def get_max_counts(spectra, energy, emin, emax, elements):
    condition = (energy > emin) & (energy < emax)
    if spectra[condition].size == 0:
        print(elements, emin, emax)
        return 0
    return np.max(spectra[condition])

In [7]:
elem_dirs = [os.path.join(datadir, d) for d in os.listdir(datadir)]
energy = su.load_xmso(os.path.join(elem_dirs[0], os.listdir(elem_dirs[0])[0]))[1]
data = dict()
for _dir in elem_dirs:
    nelem = len(os.path.basename(_dir).split('-'))
    dtype = list()
    for i in range(nelem):
        dtype += [(f"el{i}", "U2"), (f"counts{i}", float), (f"wf{i}", float)]
    dtype += [("rlth", float), ("hcth", float)]
    data_tuple = list()
    xmso_fnames = [os.path.join(_dir, f) for f in os.listdir(_dir)]
    for f in xmso_fnames:
        spectra = su.load_xmso(f)[0]
        info = su.parse_xmso_fname(f)
        elements = info.elements.split('-')
        i = 0
        elem_data = list()
        while i < len(elements):
            emean = sorted(xrdata.loc[(xrdata["Symbol"] == elements[i], "Ka1")].values)[0]/1000
            maxc = get_max_counts(spectra, energy, emean - 0.3, emean + 0.3, elements)
            elem_data += [elements[i], maxc, elements[i + 1]]
            i += 2
        data_tuple.append((*tuple(elem_data), info.reflay, info.hydro))
    data[os.path.basename(_dir)] = np.array(data_tuple, dtype = dtype)
data

{'Mn-Fe': array([('Mn', 16.1569  , 25., 'Fe', 48.9305 , 75., 3.4e-04, 0.01 ),
        ('Mn', 12.2493  , 25., 'Fe', 37.1401 , 75., 2.3e-04, 0.01 ),
        ('Mn',  7.2485  , 25., 'Fe', 21.8738 , 75., 1.2e-04, 0.001),
        ('Mn',  7.28699 , 25., 'Fe', 22.0922 , 75., 1.2e-04, 0.067),
        ('Mn', 21.5386  , 25., 'Fe', 65.4528 , 75., 5.6e-04, 0.01 ),
        ('Mn',  7.23104 , 25., 'Fe', 21.8872 , 75., 1.2e-04, 0.01 ),
        ('Mn', 26.8328  , 25., 'Fe', 82.2915 , 75., 1.0e-03, 0.01 ),
        ('Mn',  7.25866 , 25., 'Fe', 22.039  , 75., 1.2e-04, 0.056),
        ('Mn',  7.27159 , 25., 'Fe', 21.9744 , 75., 1.2e-04, 0.012),
        ('Mn', 23.3487  , 25., 'Fe', 71.1166 , 75., 6.7e-04, 0.01 ),
        ('Mn', 24.7894  , 25., 'Fe', 75.7329 , 75., 7.8e-04, 0.01 ),
        ('Mn', 19.1678  , 25., 'Fe', 58.2797 , 75., 4.5e-04, 0.01 ),
        ('Mn',  7.32548 , 25., 'Fe', 22.0715 , 75., 1.2e-04, 0.034),
        ('Mn', 25.9485  , 25., 'Fe', 79.4221 , 75., 8.9e-04, 0.01 ),
        ('Mn',  7.27395 ,

In [82]:
newdtype = data["Cr-Mn"].dtype.descr + [("rl/hc", float)]
newdata = np.empty(data["Cr-Mn"].shape, dtype = newdtype)
for n in data["Cr-Mn"].dtype.names:
    newdata[n] = data["Cr-Mn"][n]
newdata["rl/hc"] = data["Cr-Mn"]["rlth"]/data["Cr-Mn"]["hcth"]
newdata

array([('Cr', 12.1357  , 97.74, 'Mn',  1.86117 ,  2.26, 5.20e-05, 2.522e-03, 2.06185567e-02),
       ('Cr', 10.5885  , 62.37, 'Mn',  7.49037 , 37.63, 7.20e-05, 2.476e-03, 2.90791599e-02),
       ('Cr', 19.9951  , 79.81, 'Mn',  7.26709 , 20.19, 1.13e-04, 3.084e-03, 3.66407263e-02),
       ('Cr',  0.669338,  4.27, 'Mn', 13.2945  , 95.73, 5.70e-05, 1.605e-03, 3.55140187e-02),
       ('Cr', 11.6423  , 75.79, 'Mn',  4.97186 , 24.21, 6.50e-05, 1.620e-03, 4.01234568e-02),
       ('Cr',  9.89709 , 60.62, 'Mn',  7.42124 , 39.38, 6.90e-05, 1.583e-03, 4.35881238e-02),
       ('Cr',  0.418744, 23.42, 'Mn',  1.29334 , 76.58, 6.00e-06, 1.214e-03, 4.94233937e-03),
       ('Cr',  3.60053 , 39.76, 'Mn',  5.76062 , 60.24, 3.60e-05, 2.513e-03, 1.43255074e-02),
       ('Cr',  1.61067 , 25.57, 'Mn',  4.74308 , 74.43, 2.40e-05, 2.148e-03, 1.11731844e-02),
       ('Cr',  2.86503 , 46.3 , 'Mn',  3.5918  , 53.7 , 2.40e-05, 1.230e-03, 1.95121951e-02),
       ('Cr',  2.53437 , 20.53, 'Mn',  9.75588 , 79.47, 4.90

In [88]:
plt.figure(1)
for w in np.unique(np.round(newdata["wf1"],2)):
    condition = np.round(newdata["wf1"],2) == w
    d = np.sort(newdata[condition], order = "rl/hc")
    plt.plot(d["rl/hc"], d["counts1"], label = f'{d["el0"][0]} {d["wf0"][0]}% {d["el1"][0]} {d["wf1"][0]}%')
plt.xlabel("reference layer thickness / hydrocerussite thickness ratio")
plt.ylabel("max counts")
plt.legend()
plt.title(r"K-Ca counts in Ca-K$\alpha$1 en. range vs rlthickness/hcthickness ratio")
plt.show()

In [13]:
el = 'Mn-Fe'
newdata = data[el][data[el]["rlth"] == 1.2e-04]
plt.figure(2)
d = np.sort(newdata, order = "hcth")
plt.plot(d["hcth"], d["counts0"])
plt.show()